In [ ]:
import os

# Set CUDA_VISIBLE_DEVICES to a specific GPU, e.g., '0' for the first GPU
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# To disable CUDA from seeing any GPUs, set it to an empty string
# os.environ['CUDA_VISIBLE_DEVICES'] = ''

# # TODO: set your huggingface cache directory. For dgxrise cluster, download large models to raid/ instead of home/ directory
# os.environ["HF_HOME"] = ""

In [ ]:
from langchain.chains.sequential import SequentialChain, SimpleSequentialChain
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from datetime import datetime
import torch
from transformers import AutoModel, pipeline, AutoTokenizer, AutoModelForCausalLM
import pandas as pd


In [ ]:
def init_model():


    # TODO: set your huggingface access token
    access_token = ''

    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=access_token)

    model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                                 device_map='auto',
                                                 torch_dtype=torch.float16,
                                                 load_in_4bit=True,
                                                 bnb_4bit_quant_type="nf4",
                                                 bnb_4bit_compute_dtype=torch.float16,
                                                 token=access_token)

    pipe = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    torch_dtype=torch.float16,
                    device_map="auto",
                    max_new_tokens=5000,
                    do_sample=True,
                    top_k=1,
                    num_return_sequences=1,
                    eos_token_id=tokenizer.eos_token_id
                    )

    llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature': 0.7, 'max_length': 5000, 'top_k': 1})

    return llm

In [ ]:
llama2 = init_model()

In [ ]:
subevent_prompt_template = """
**Article Text:**
'On December 29, 2007, a powerful extratropical disturbance developed over eastern Siberia, which emerged into the northwest Pacific, and moved to the Aleutian Islands as a well-defined low by January 3. On January 2, another extratropical disturbance developed within a well-defined trough of low pressure, in the northeast Pacific Ocean. On January 3, the 1st storm system approached the northern part of the West Coast, powered by a Pineapple Express atmospheric river, resulting in heavy downpours of rain and strong gusts across much of the Western Seaboard. Late on January 3, the 2nd storm system in the Gulf of Alaska split, with the 3rd, new storm to the south usurping the majority of the moisture of the 2nd system.'

 The article above described a climate change event that consists of the following subevents, each subevent is identified by a unique combination of date and location.
```
[
  {{
    "date": "December 29, 2007",
    "location": "Eastern Siberia",
    "peril(s)": ["Extratropical disturbance"]
  }},
  {{
    "date": "January 2",
    "location": "Northeast Pacific Ocean",
    "peril(s)": ["Extratropical disturbance"]
  }},
  {{
    "date": "January 3",
    "location": "Northern part of the West Coast",
    "peril(s)": ["Heavy rain", "Strong gusts", "Pineapple Express atmospheric river"]
  }},
  {{
    "date": "Late on January 3",
    "location": "Gulf of Alaska",
    "peril(s)": ["Storm split", "Moisture usurpation"]
  }}
]
```
**Article Text:**
{article}

The article above described a climate change event that consists of the following subevents, each subevent is identified by a unique combination of date and location.
```
[
  {{
    "date": 
"""

In [ ]:
wiki_input = """The 2008 Queensland storms were a series of three thunderstorms that struck South East Queensland, Australia. The first storm hit on 16 November 2008 and was followed by two further storms on 19 and 20 November. The storms resulted in 2 fatalities.

Background
First incident
Winds reached 130 kilometres (81 mi)/h,[2] equivalent to a Category 3 cyclone. Electricity was cut to about 230,000 homes and about 4,000 homes were left in need of serious repair, and at least 30 houses were beyond repair. Some of the worst hit areas included The Gap, Kenmore, Arana Hills and Albany Creek. Forecasters at the Bureau of Meteorology rated the storm the worst to hit south-east Queensland since 2004 & The Biggest to The South east Queensland region since 1985.[3]

Second incident
Torrential rain affected areas of Brisbane, Ipswich and Toowoomba, with rainfall reaching more than 250 mm in some locations. The Ipswich and Marburg areas were the worst affected, whilst four homes in inner-city Paddington were unroofed. The Inner City Bypass was flooded and forced to close, as was the Moggill Ferry.

Third incident
Blackwater, near Emerald in Central Queensland, was hit hard by the third storm, with 20 houses sustaining roof damage and up to 100 more were damaged in some way. Gregory MP Vaughan Johnson said the storm was terrifying. He said he was driving to an appointment when the storm slammed into Blackwater, forcing him to turn back to Emerald. Hail was described as the size of golf balls and witnesses saw a lot of damage."""

In [ ]:
subevent_prompt = PromptTemplate(template=subevent_prompt_template, input_variables=["article"])
subevent_chain = LLMChain(llm=llama2, prompt=subevent_prompt, output_key="subevent")

In [ ]:
# subevent_response = subevent_chain.run(wiki_input)
# print(subevent_response)

In [ ]:
location_prompt_template="""
**Article Text:**
'On December 29, 2007, a powerful extratropical disturbance developed over eastern Siberia, which emerged into the northwest Pacific, and moved to the Aleutian Islands as a well-defined low by January 3. On January 2, another extratropical disturbance developed within a well-defined trough of low pressure, in the northeast Pacific Ocean. On January 3, the 1st storm system approached the northern part of the West Coast, powered by a Pineapple Express atmospheric river, resulting in heavy downpours of rain and strong gusts across much of the Western Seaboard. Late on January 3, the 2nd storm system in the Gulf of Alaska split, with the 3rd, new storm to the south usurping the majority of the moisture of the 2nd system.'

The article above mentioned the following locations:
- "estern Siberia"
- "northwest Pacific"
- "Aleutian Islands"
- "northeast Pacific Ocean"
- "northern part of the West Coast"
- "western Seaboard"
- "Gulf of Alaska"


**Article Text:**
{article}

The article above mentioned the following locations:
- "
"""

In [ ]:
location_prompt = PromptTemplate(template=location_prompt_template, input_variables=["article"])
location_chain = LLMChain(llm=llama2, prompt=location_prompt, output_key="location")

In [ ]:
# location_response = location_chain.run(wiki_input)
# print(location_response)

In [ ]:
subevent_cont_prompt_template="""
**Article Text:**
'On December 29, 2007, a powerful extratropical disturbance developed over eastern Siberia, which emerged into the northwest Pacific, and moved to the Aleutian Islands as a well-defined low by January 3. On January 2, another extratropical disturbance developed within a well-defined trough of low pressure, in the northeast Pacific Ocean. On January 3, the 1st storm system approached the northern part of the West Coast, powered by a Pineapple Express atmospheric river, resulting in heavy downpours of rain and strong gusts across much of the Western Seaboard. Late on January 3, the 2nd storm system in the Gulf of Alaska split, with the 3rd, new storm to the south usurping the majority of the moisture of the 2nd system.'

The article above described a climate change event that consists of the following subevents, with a location span of
- "estern Siberia"
- "northwest Pacific"
- "Aleutian Islands"
- "northeast Pacific Ocean"
- "northern part of the West Coast"
- "western Seaboard"
- "Gulf of Alaska"

```
[
  {{
    "date": "December 29, 2007",
    "location": "Eastern Siberia",
    "peril(s)": ["Extratropical disturbance"]
  }},
  {{
    "date": "January 2",
    "location": "Northeast Pacific Ocean",
    "peril(s)": ["Extratropical disturbance"]
  }},
  {{
    "date": "January 3",
    "location": "Northern part of the West Coast",
    "peril(s)": ["Heavy rain", "Strong gusts", "Pineapple Express atmospheric river"]
  }},
  {{
    "date": "Late on January 3",
    "location": "Gulf of Alaska",
    "peril(s)": ["Storm split", "Moisture usurpation"]
  }}
]
```
**Article Text:**
{article}

The article above described a climate change event that consists of the following subevents, with a location span of
- "{location}
```
[
  {{
    "date": 
"""

In [ ]:
subevent_cont_prompt = PromptTemplate(template=subevent_cont_prompt_template, input_variables=["article", "location"])
subevent_cont_chain = LLMChain(llm=llama2, prompt=subevent_cont_prompt, output_key="subevent_cont")
sequential_chain = SequentialChain(chains=[location_chain, subevent_cont_chain], input_variables=[
                                   "article"], output_variables=["location", "subevent_cont"])

In [ ]:
results = sequential_chain({"article": wiki_input})
print(results["location"])
print("="*20)
print(results["subevent_cont"])